In [ ]:
## reset specific variables (replace regular_expression by the variables of interest)
#%reset_selective <regular_expression>

# reset all variables
%reset -f

In [189]:
## Importing libraries

from datetime import datetime, date, timedelta
from array import *
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from pylab import savefig
import seaborn as sns
import pandas as pd
import csv
import json
from pymongo import MongoClient
from mongoengine import *

%matplotlib inline

In [20]:
## Creating/Connecting Mongo DB instances

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = "mongodb+srv://<jgu>:<123>@<cluster-jgu>.mongodb.net/SMARTAttributesFilter"

connect(db='SMARTAttributesFilter', alias='SMARTAttributesFilter_alias')

connect(db='FailuresAppsLocation', alias='FailuresAppsLocation_alias')

connect(db='SMARTAtt_FailuresAppsLocation', alias='SMARTAtt_FailuresAppsLocation_alias')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [21]:
## Setting document schema

class SMARTAtt(Document):
     disk_id = FloatField(required=False, default='0')
     timestamp = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     r_sectors = FloatField(required=False, default='0')
     u_errors = FloatField(required=False, default='0')
     p_failedA = FloatField(required=False, default='0')
     p_failedB = FloatField(required=False, default='0')
     e_failedA = FloatField(required=False, default='0')
     e_failedB = FloatField(required=False, default='0')
     n_b_written = FloatField(required=False, default='0')
     n_b_read = FloatField(required=False, default='0')
     meta = {'db_alias': 'SMARTAttributesFilter_alias'}

class FailuresAppsLocation(Document):
     disk_id = FloatField(required=False, default='0')
     failure_time = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     model_y = StringField(required=False, default='0')
     app = StringField(required=False, default='0')
     node_id = FloatField(required=False, default='0')
     rack_id = FloatField(required=False, default='0')
     machine_room_id = FloatField(required=False, default='0')
     meta = {'db_alias': 'FailuresAppsLocation_alias'}

class SMARTAtt_FailuresAppsLocation(Document):
     smart_att = ReferenceField(SMARTAtt)
     failures_app_location = ReferenceField(FailuresAppsLocation)
     meta = {'db_alias': 'SMARTAtt_FailuresAppsLocation_alias'}

In [ ]:
# Closing the connection to the DB

disconnect(alias='SMARTAttributesFilter_alias')

disconnect(alias='FailuresAppsLocation_alias_alias')

disconnect(alias='SMARTAtt_FailuresAppsLocation_alias')

In [168]:
## Some queries in Mongoengine

# Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects()

# Collecting the applications, except for "none"
apps = FailuresAppsLocation.objects().distinct("app")
arrayForDataPerApp = np.empty(0)


#a = np.empty((len(apps),0), FailuresAppsLocation)
TdataPerApp = FailuresAppsLocation.objects(app='none').filter()

#for doc in FailuresAppsLocation.objects:
 #   arrayForDataPerApp = np.append(arrayForDataPerApp, pd.read_json(doc.to_json()))


for app_c in apps:
   dataPerApp = FailuresAppsLocation.objects(app=app_c).filter()
   #dataPerAppToList = list(dataPerApp)
   #dataPerAppDf = pd.DataFrame.from_dict(dataPerAppToList)
     
   for doc in dataPerApp:
      arrayForDataPerApp = np.append(arrayForDataPerApp, doc.to_json())



#for y in FailuresAppsLocation.objects:
 #   print(y.disk_id)

#print(a[1])
#The collection size
#len(failuresAppsLocationTeste)

#Return the number of different items of some specific field
#result = failuresAppsLocationTeste.distinct("rack_id")
#len(result)

# Query if you know something about the document
#testando = FailuresAppsLocation.objects(disk_id="33722").get()

# Printing
#testando.disk_id

# Query if you know something about the document
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter().limit(20)
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter()

# Printing
#for i in SMARTAttributesTest:
#    print(i.timestamp)
    
# Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects() 

# Deleting all collection
#failuresAppsLocationTeste.delete() 
  
 #Deleting all collection
#for i in failuresAppsLocationTeste:
 # i.delete()

In [197]:
## Space for small tests

#df = pd.DataFrame()

#for i in FailuresAppsLocation.objects:
 #   df = pd.concat([df,pd.DataFrame(pd.read_json(i.to_json(), orient='_id'))])    
#df

failuresLocationDocuments = FailuresAppsLocation.objects()

jSon_failuresLocationDocumentsJson = json.loads(failuresLocationDocuments.to_json())
df_failuresLocationDocumentsJson = pd.DataFrame.from_dict(jSon_failuresLocationDocumentsJson) 
df_failuresLocationDocumentsJson


,_id,disk_id,failure_time,model_x,model_y,app,node_id,rack_id,machine_room_id
0,{'$oid': '6266c2e8718b63c8dff976fb'},33722.0,{'$date': 1514920532000},MA2,A2,RM,66688.0,11488.0,455.0
1,{'$oid': '6266c2e8718b63c8dff976fc'},58337.0,{'$date': 1514933116000},MA2,A2,RM,114833.0,9859.0,455.0
2,{'$oid': '6266c2e8718b63c8dff976fd'},26378.0,{'$date': 1514949824000},MA1,A3,none,145735.0,6860.0,127.0
3,{'$oid': '6266c2e8718b63c8dff976fe'},39876.0,{'$date': 1514950167000},MA1,A3,RM,217964.0,17596.0,92.0
4,{'$oid': '6266c2e8718b63c8dff976ff'},199348.0,{'$date': 1514955783000},MC1,C1,NAS,237451.0,22765.0,137.0
...,...,...,...,...,...,...,...,...,...
16300,{'$oid': '6266c2f4718b63c8dff9b6a7'},12463.0,{'$date': 1577808965000},MC2,C2,none,124346.0,4864.0,56.0
16301,{'$oid': '6266c2f4718b63c8dff9b6a8'},10186.0,{'$date': 1577820484000},MC2,C2,RM,101995.0,398.0,647.0
16302,{'$oid': '6266c2f4718b63c8dff9b6a9'},18144.0,{'$date': 1577820770000},MC1,C1,NAS,20886.0,5094.0,149.0
16303,{'$oid': '6266c2f4718b63c8dff9b6aa'},99516.0,{'$date': 1577829456000},MA2,A2,RM,195437.0,16394.0,565.0


In [ ]:
# CENARIO 1

## WRITE IOPS

fig = sns.catplot(x="capacity", y="iops_total", hue="object_size", data=dataset_write_cenUm,
               row="pattern", col="work_outstd", kind="bar", ci=90, palette="Blues_d", aspect=0.9, height=4.5
                  , legend_out = True, margin_titles = True)

fig.set_axis_labels("Capacity", "IOPS")
fig.set_xticklabels(["80GB HDD", "500GB HDD", "1TB HDD", "1TB WDHDD", "120GB SSD", "Hybrid"])

plt.savefig('write_iops.pdf', dpi=1200)